# Exercise 2

**Please Note**: We updated the requirements.txt

Please install the new requirements before editing this exercise.

## Import packages

In [1]:
import os

from vll.utils.download import download_mnist
import numpy as np
import matplotlib.pyplot as plt

import skimage
import skimage.io

import torch
import torch.nn.functional as F
from torchvision import transforms

from models.mnist.simple_cnn import Net

## Task 1
(2 points)

In this task, you will learn some basic tensor operations using the PyTorch library.

Reference for torch: https://pytorch.org/docs/stable/torch.html

In [6]:
# Create a numpy array that looks like this: [0, 1, 2, ..., 19]
arr = np.arange(20)

# Convert the numpy array to a torch tensor
tensor = torch.from_numpy(arr)
print(tensor)

# Create a tensor that contains random numbers.
# It should have the same size like the numpy array.
# Multiply it with the previous tensor.
rand_tensor = torch.rand_like(torch.from_numpy(arr), dtype=torch.float)
tensor = rand_tensor*tensor
print(tensor)

# Create a tensor that contains only 1s.
# It should have the same size like the numpy array.
# Substract it from the previous tensor.
tensor = tensor - torch.ones_like(torch.from_numpy(arr))
print(tensor)

# Get the 5th element using a index.
element = tensor[4] 
print(element)

# Create a tensor that contains only 0s.
# It should have the same size like the numpy array.
# Multiply it with the previous tensor without any assignment (in place).
tensor_zeros = torch.zeros_like(torch.from_numpy(arr), dtype=torch.float)
tensor.mul_(tensor_zeros)


tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19], dtype=torch.int32)
tensor([ 0.0000,  0.1304,  0.8244,  0.4302,  2.4022,  0.4153,  0.5419,  4.7282,
         0.5169,  0.6433,  8.4377,  6.9649, 10.4643,  4.4864, 12.5367,  2.7041,
        10.4040, 16.3172,  0.4987,  9.9437])
tensor([-1.0000, -0.8696, -0.1756, -0.5698,  1.4022, -0.5847, -0.4581,  3.7282,
        -0.4831, -0.3567,  7.4377,  5.9649,  9.4643,  3.4864, 11.5367,  1.7041,
         9.4040, 15.3172, -0.5013,  8.9437])
tensor(1.4022)


tensor([-0., -0., -0., -0., 0., -0., -0., 0., -0., -0., 0., 0., 0., 0., 0., 0., 0., 0., -0., 0.])

In [7]:
# Load the image from the last exercise as RGB image.
image = skimage.io.imread('./data/pepo.jpg')

# Convert the image to a tensor
image = transforms.Compose([transforms.ToTensor()])(image)

# Print its shape
print(image.shape)

# Flatten the image
image = torch.flatten(image) 
print(len(image))

# Add another dimension resulting in a 1x78643 tensor
image = image.unsqueeze(0)
print(image.shape)

# Revert the last action
image = image.squeeze(0)
print(image.shape)

# Reshape the tensor, so that it has the original 2D dimensions
image = image.reshape(3, 512, 512)
print(image.shape)

# Calculate the sum, mean and max of the tensor
print(torch.sum(image))
print(torch.mean(image))
print(torch.max(image))

torch.Size([3, 512, 512])
786432
torch.Size([1, 786432])
torch.Size([786432])
torch.Size([3, 512, 512])
tensor(358450.3750)
tensor(0.4558)
tensor(1.)


## Task 2
(2 points)

Use Autograd to perform operations on a tensor and output then gradients.

In [11]:
# Create a random 2x2 tensor which requires gradients
x = torch.rand(2,2, requires_grad=True) 
print(x)

# Create another tensor by adding 2.0
y = x+2.0
print(y)

# Create a third tensor z = y^2
z = y**2
print(z)

# Compute out as the mean of values in z
out = torch.mean(z)
print(out)

# Perform back propagation on out
out.backward()

# Print the gradients dout/dx
print(x.grad)

# Create a copy of y whithout gradients
y2 = y.detach()
print(y2.requires_grad)

# Perform the mean operation on z
# with gradients globally disabled
with torch.no_grad():
    print(torch.mean(z))


tensor([[0.8453, 0.8624],
        [0.9401, 0.9901]], requires_grad=True)
tensor([[2.8453, 2.8624],
        [2.9401, 2.9901]], grad_fn=<AddBackward0>)
tensor([[8.0960, 8.1933],
        [8.6444, 8.9407]], grad_fn=<PowBackward0>)
tensor(8.4686, grad_fn=<MeanBackward0>)
tensor([[1.4227, 1.4312],
        [1.4701, 1.4950]])
False
tensor(8.4686)


## Task 3
(3 points)

Implement a Dataset class for MNIST.

In [ ]:
# We first download the MNIST dataset
download_mnist()

In [ ]:
class MNIST:
    """
    Dataset class for MNIST
    """

    def __init__(self, root, transform=None):
        """
        root -- path to either "training" or "testing"
        
        transform -- transform (from torchvision.transforms)
                     to be applied to the data
        """
        # save transforms
        self.transform = transform
        
        # TODO: create a list of all subdirectories (named like the classes) 
        #       within the dataset root
        
        
        # TODO: create a list of paths to all images
        #       with the ground truth label
        
    
    def __len__(self):
        """
        Returns the lenght of the dataset (number of images)
        """
        # TODO: return the length (number of images) of the dataset
        

    def __getitem__(self, index):
        """
        Loads and returns one image as floating point numpy array
        
        index -- image index in [0, self.__len__() - 1]
        """
        # TODO: load the ith image as an numpy array (dtype=float32)
        
        
        # TODO: apply transforms to the image (if there are any)
        
        
        # TODO: return a tuple (transformed image, ground truth)
       

## Task 4
(3 points)

You can now load a pretrained neural network model we provide.
Your last task is to run the model on the MNIST test dataset, plot some example images with the predicted labels and compute the prediction accuracy.

In [ ]:
def validate(model, data_loader):
    # TODO: Create a 10x10 grid of subplots
   
    
    model.eval()
    correct = 0 # count for correct predictions
    
    with torch.no_grad():
        for i, item in enumerate(data_loader):
            # TODO: unpack item into image and ground truth
            #       and run network on them
            
            
            # TODO: get class with highest probability
            
            
            # TODO: check if prediction is correct
            #       and add it to correct count
            
            
            # plot the first 100 images
            if i < 100:
                # TODO: compute position of ith image in the grid
                
                
                # TODO: convert image tensor to numpy array
                #       and normalize to [0, 1]
                
                
                # TODO: make wrongly predicted images red
                
                
                # TODO: disable axis and show image
                
                
                # TODO: show the predicted class next to each image
                
            
            elif i == 100:
                plt.show()
    
    # TODO: compute and print the prediction accuracy in percent
    

# create a DataLoader using the implemented MNIST dataset class
data_loader = torch.utils.data.DataLoader(
    MNIST('data/mnist/testing',
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
    ])),
    batch_size=1, shuffle=True)

# create the neural network
model = Net()

# load the statedict from 'models/mnist/simple_cnn.pt'
model.load_state_dict(torch.load('models/mnist/simple_cnn.pt'))

# validate the model
validate(model, data_loader)